In [1]:
# --- Notebook: image_server.ipynb ---
# Cell 1: Install deps
%pip -q install fastapi uvicorn pillow nest-asyncio


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Cell 2: Imports + helpers
from __future__ import annotations

import base64
import io
import re
from typing import Tuple

from fastapi import FastAPI
from pydantic import BaseModel, Field
from PIL import Image, ImageDraw, ImageFont




In [3]:
# IMAGE Functions

def _clamp_u8(x: int) -> int:
    return 0 if x < 0 else 255 if x > 255 else x


def _parse_color(name: str) -> Tuple[int, int, int, int]:
    """
    Supports:
      - 'red', 'green', 'blue', 'white', 'black', 'transparent'
      - '#RRGGBB' or '#RRGGBBAA'
      - 'rgba(r,g,b,a)' where a is 0..255 or 0..1
    """
    s = (name or "").strip().lower()

    named = {
        "red": (255, 0, 0, 255),
        "green": (0, 255, 0, 255),
        "blue": (0, 0, 255, 255),
        "white": (255, 255, 255, 255),
        "black": (0, 0, 0, 255),
        "transparent": (0, 0, 0, 0),
    }
    if s in named:
        return named[s]

    if s.startswith("#"):
        hx = s[1:]
        if len(hx) == 6:
            r = int(hx[0:2], 16); g = int(hx[2:4], 16); b = int(hx[4:6], 16)
            return (r, g, b, 255)
        if len(hx) == 8:
            r = int(hx[0:2], 16); g = int(hx[2:4], 16); b = int(hx[4:6], 16); a = int(hx[6:8], 16)
            return (r, g, b, a)

    m = re.match(r"rgba\(\s*(\d+)\s*,\s*(\d+)\s*,\s*(\d+)\s*,\s*([0-9.]+)\s*\)", s)
    if m:
        r = _clamp_u8(int(m.group(1)))
        g = _clamp_u8(int(m.group(2)))
        b = _clamp_u8(int(m.group(3)))
        a_raw = float(m.group(4))
        a = _clamp_u8(int(a_raw * 255.0)) if a_raw <= 1.0 else _clamp_u8(int(a_raw))
        return (r, g, b, a)

    # fallback: mid grey
    return (128, 128, 128, 255)


def _render_rgba_png(size: int, contents: str) -> bytes:
    """
    Minimal "contents" grammar:
      - "solid:<color>"                 e.g. "solid:#112233" or "solid:rgba(10,20,30,0.5)"
      - "checker:<color1>,<color2>,<n>" e.g. "checker:black,white,8" (n=tiles per side)
      - "text:<message>"               e.g. "text:hello"
    Defaults to solid grey if unrecognized.
    """
    s = (contents or "").strip()

    img = Image.new("RGBA", (size, size), (0, 0, 0, 0))
    draw = ImageDraw.Draw(img)

    if s.lower().startswith("solid:"):
        color = _parse_color(s.split(":", 1)[1])
        draw.rectangle((0, 0, size, size), fill=color)

    elif s.lower().startswith("checker:"):
        _, rest = s.split(":", 1)
        parts = [p.strip() for p in rest.split(",")]
        c1 = _parse_color(parts[0]) if len(parts) > 0 else (0, 0, 0, 255)
        c2 = _parse_color(parts[1]) if len(parts) > 1 else (255, 255, 255, 255)
        n  = int(parts[2]) if len(parts) > 2 and parts[2].isdigit() else 8
        n  = max(1, min(256, n))
        tile = max(1, size // n)

        for y in range(0, size, tile):
            for x in range(0, size, tile):
                use = c1 if ((x // tile) + (y // tile)) % 2 == 0 else c2
                draw.rectangle((x, y, x + tile - 1, y + tile - 1), fill=use)

    elif s.lower().startswith("text:"):
        msg = s.split(":", 1)[1]
        draw.rectangle((0, 0, size, size), fill=(18, 18, 18, 255))
        try:
            font = ImageFont.load_default()
        except Exception:
            font = None
        draw.text((8, 8), msg[:200], fill=(240, 240, 240, 255), font=font)

    else:
        draw.rectangle((0, 0, size, size), fill=(128, 128, 128, 255))

    # encode to PNG bytes
    buf = io.BytesIO()
    img.save(buf, format="PNG")
    return buf.getvalue()


def _png_bytes_to_b64(png_bytes: bytes) -> str:
    return base64.b64encode(png_bytes).decode("ascii")


In [4]:
# NETWORK Functions

# Cell 4: FastAPI app + request/response models
app = FastAPI(title="RGBA PNG generator", version="1.0.0")

class ImageRequest(BaseModel):
    size: int = Field(..., ge=1, le=2048, description="Image width/height (square).")
    contents: str = Field(..., description="Content spec string.")


class ImageResponse(BaseModel):
    width: int
    height: int
    mime: str
    rgba_png_base64: str


# Test endpoint
@app.get("/health")
def health():
    return {"status": "ok"}


# Register the route with the app
@app.post("/render", response_model=ImageResponse)
def render(req: ImageRequest) -> ImageResponse:
    try:
        png = _render_rgba_png(req.size, req.contents)
        return ImageResponse(
            width=req.size,
            height=req.size,
            mime="image/png",
            rgba_png_base64=_png_bytes_to_b64(png),
        )
    except Exception as e:
        import traceback
        traceback.print_exc()
        raise




In [5]:
# Cell 5: Run the server in the current notebook
# Using threading with proper app reference
import threading
import uvicorn
import time

def run_server():
    # Run uvicorn with the app object (not by import string)
    uvicorn.run(app, host="127.0.0.1", port=8000, log_level="info", use_colors=False)

# Start server in a daemon thread
server_thread = threading.Thread(target=run_server, daemon=True)
server_thread.start()

time.sleep(3)  # Give server time to start
print("Server started on http://127.0.0.1:8000")
print("The server is running in a background thread.")
print("Note: The server will stop when you restart the kernel.")


INFO:     Started server process [21428]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Server started on http://127.0.0.1:8000
The server is running in a background thread.
Note: The server will stop when you restart the kernel.


INFO:     127.0.0.1:55490 - "POST /render HTTP/1.1" 200 OK
